In [1]:
tstring = '8A004A801A8002F478'
tests = {'8A004A801A8002F478':16,'620080001611562C8802118E34':12,'C0015000016115A2E0802F182340':23,'A0016C880162017C3686B18A3D4780':31}

In [2]:
lookup = {'0' : '0000','1' : '0001','2' : '0010','3' : '0011','4' : '0100','5' : '0101','6' : '0110','7' : '0111','8' : '1000','9' : '1001','A' : '1010','B' : '1011','C' : '1100','D' : '1101','E' : '1110','F' : '1111'}
    
def to_binary_string(s):
    out = ''
    for c in s:
        out+=lookup[c]
    return out


to_binary_string(tstring)

'100010100000000001001010100000000001101010000000000000101111010001111000'

In [3]:
to_binary_string('D2FE28')

'110100101111111000101000'

In [5]:
110100101111111000101000
VVVTTTAAAAABBBBBCCCCC
Below each bit is a label indicating its purpose:

The three bits labeled V (110) are the packet version, 6.
The three bits labeled T (100) are the packet type ID, 4, which means the packet is a literal value.
The five bits labeled A (10111) start with a 1 (not the last group, keep reading) and contain the first four bits of the number, 0111.
The five bits labeled B (11110) start with a 1 (not the last group, keep reading) and contain four more bits of the number, 1110.
The five bits labeled C (00101) start with a 0 (last group, end of packet) and contain the last four bits of the number, 0101.
The three unlabeled 0 bits at the end are extra due to the hexadecimal representation and should be ignored.



SyntaxError: invalid syntax (<ipython-input-5-e3932cf10836>, line 3)

In [4]:
def value(bs):
    val = 0
    for e, b in enumerate(bs[::-1]):
        if b == '1':
            val+=2**e
    return val


In [6]:
def literal_packet(rest):
    ver_num = ''
    while True:
        ver_num += rest[1:5]
        x = rest[0]
        rest = rest[5:]
        if x =='0':
            break
    return value(ver_num),rest

literal_packet('0101001000100100')

(10, '01000100100')

In [28]:
f = open('input.txt','r')
bstring = f.read().replace('\n','')


a = []
def packet(trans):
    pversion,ptype,rest = int(trans[:3],2),int(trans[3:6],2),trans[6:]
    a = []
    versions,result = 0,0
    if ptype == 4:
        result, rest = literal_packet(rest)
    elif ptype != 4:
        x = rest[0]
        rest = rest[1:]
        subpackets = []
        if x == '0':        #next 15 bits are a number that represents the total length in bits of the sub-packets contained by this packet.
            subpacked_length = int(rest[:15],2)
            rest = rest[15:]
            estimatedrestlength = len(rest) - subpacked_length
            while len(rest)>estimatedrestlength:
                subpacket, rest = packet(rest)
                for sub in subpacket: subpackets.append(sub)
        elif x == '1':       #next 11 bits are a number that represents the number of sub-packets immediately contained by this packet
            subpacked_count= int(rest[:11],2)
            rest = rest[11:]
            for _ in range(subpacked_count):
                subpacket, rest = packet(rest)
                for sub in subpacket: subpackets.append(sub)
        for x in subpackets:
            versions += x[0]
        subresults=[]
        if ptype ==0:  #Addition
            for x in subpackets:
                result +=x[2]
        elif ptype ==1:
            result = 1
            for x in subpackets:
                result *=x[2]
        elif ptype ==2:
            for x in subpackets:
                subresults += [x[2]]
            result = min(subresults)
        elif ptype ==3:
            for x in subpackets:
                subresults += [x[2]]
            result = max(subresults)
        elif ptype ==5:
            if subpackets[0][2]>subpackets[1][2]: 
                result = 1 
        elif ptype ==6:
            if subpackets[0][2]<subpackets[1][2]: 
                result = 1 
        elif ptype ==7:
            if subpackets[0][2]==subpackets[1][2]: 
                result = 1 
    a.append((pversion+versions,ptype,result))
    return a , rest

packets, rest = packet(to_binary_string(bstring))
print('part1',packets[-1][0])
print('part2',packets[-1][2])

part1 947
part2 660797830937


In [8]:
b = to_binary_string('38006F45291200')
packet_version = value(b[0:3])
packet_type = value(b[3:6])
if packet_type == 4:
    a_reg = b[6:6+5]
    b_reg = b[6+5:6+5+5]
    c_reg = b[6+5+5:6+5+5+5]

    if a_reg[0] == '1':
        a_val = a_reg[1:]
    if b_reg[0] == '1':
        b_val = b_reg[1:]
    if c_reg[0] == '0':
        c_val = c_reg[1:]


    value(a_val+b_val+c_val)
else:
    length_type_id = b[5]
    
    if b[5]:
        print('I')
    if length_type_id:
        nextbits = 15
    else:
        nextbits=11
    L = value(b[6:6+nextbits+1])
    a_reg = b[6+nextbits+1:6+nextbits+1+11]
    b_reg = b[6+nextbits+1+11:6+nextbits+1+L]
    
    print(a_reg)
    _,_,a_val = literal_packet(a_reg)
    _,_,b_val = literal_packet(b_reg)
    print(L,a_val,b_val)
    
    print(packet_type)



I
11010001010
27 10 20
6


True

In [311]:
to_binary_string('A0016C880162017C3686B18A3D4780')[7+15+1:91+7+15+1]

'0100010000000000101100010000000010111110000110110100001101011000110001010001111010100011110'

In [312]:
to_binary_string('A0016C880162017C3686B18A3D4780')

'101000000000000101101100100010000000000101100010000000010111110000110110100001101011000110001010001111010100011110000000'

# bib = to_binary_string('8A004A801A8002F478')
bib

In [264]:
bib[7:7+11]

'00000000001'

In [265]:
len('001010100000000001101010000000000000101111010001111')

51

In [266]:
to_binary_string('8A004A801A8002F478')

'100010100000000001001010100000000001101010000000000000101111010001111000'

In [267]:
for k,v in tests.items():
    mybs = to_binary_string(k)
    p = process(mybs,0)
    if v==p:
        print('pass')
    else:
        print('failed for:',k,v,p)


btype 4
number_sub_packets 1
btype 1
number_sub_packets 1
btype 5
11010001111 11
btype 6
failed for: 8A004A801A8002F478 16 25
btype 3
number_sub_packets 2
btype 0
btype 0
number_sub_packets 6
btype 0
00000010110
btype 0
00000010110
btype 0
00000010110
btype 0
00000010110
btype 0
00000010110
btype 0
00000010110
failed for: 620080001611562C8802118E34 12 3
btype 6
00000000000 11
btype 0
00000010110 11
btype 0
number_sub_packets 6
btype 0
00000010110
btype 0
00000010110
btype 0
00000010110
btype 0
00000010110
btype 0
00000010110
btype 0
00000010110
00010001010 11
btype 0
11010001011 11
btype 6
10000010000 11
btype 4
number_sub_packets 0
00000101111 11
btype 0
 0
btype 0


IndexError: string index out of range

In [268]:
len('00000010110')

11

In [174]:
mybs = to_binary_string('38006F45291200')
mybs[0:7]
literal_packet(mybs[8+15:8+15+27])

(5, 0, 17428)

In [175]:
mybs[8+15:8+15+27]

'101000101001010010001001000'

In [152]:
len(mybs[7:7+11])

11

In [130]:
value('00000000010')

2

In [122]:
len('101100010110')

12

In [213]:
for k,v in tests.items():
    mybs = to_binary_string(k)
    p = process(mybs,0)
    if v==p:
        print('pass')
    else:
        print('failed for:',k,v,p)

number_sub_packets 1
number_sub_packets 0
failed for: 8A004A801A8002F478 16 5
number_sub_packets 2
11 22
number_sub_packets 6


IndexError: string index out of range

In [82]:
to_binary_string('620080001611562C8802118E34')

'01100010000000001000000000000000000101100001000101010110001011001000100000000010000100011000111000110100'

In [53]:
8A004A801A8002F478 represents an operator packet (version 4) which contains an 
operator packet (version 1) which contains an operator packet (version 5)
which contains a literal value (version 6); this packet has a version sum of 16.

''

In [ ]:
11010001010

In [46]:
len('AAAAAAAAAAABBBBBBBBBBBBBBBB')

literal_packet('11010001010')
#literal_packet('0101001000100100')

(6, 4, 10)

In [ ]:
00111000000000000110111101000101001010010001001000000000
VVVTTTILLLLLLLLLLLLLLLAAAAAAAAAAABBBBBBBBBBBBBBBB

In [71]:
len('AAAAAAAAAAABBBBBBBBBBBBBBBB')

27

In [72]:
len('AAAAAAAAAAA')

11

In [ ]:
If the length type ID is 0, then the next 15 bits are a number that represents the total length in bits of the sub-packets contained by this packet.
If the length type ID is 1, then the next 11 bits are a number that represents the number of sub-packets immediately contained by this packet.


In [31]:
nextbits

15

In [32]:
b[6:6+nextbits]

'000000000001101'